In [1]:
import sqlite3
import pandas as pd

# List of database filenames
database_files = ["Treasury_Yields_data.db", "CoT_data.db"]  # Add your database filenames

# Create an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate through each database
for db_file in database_files:
    # Create a connection to the database
    conn = sqlite3.connect(db_file)

    # Get a list of all tables in the database
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = conn.execute(tables_query).fetchall()

    # Filter tables that contain the phrase "weekly_percent_change"
    relevant_tables = [table[0] for table in tables if "weekly_percent_change" in table[0]]

    # Iterate through relevant tables
    for table_name in relevant_tables:
        # Read the table into a DataFrame
        query = f"SELECT * FROM '{table_name}';"
        df = pd.read_sql_query(query, conn, index_col='Date', parse_dates='Date')

        # Extract necessary columns (you may adjust this based on your data structure)
        #relevant_columns = ["column1", "column2", ...]  # Replace with your relevant column names
        #df = df[relevant_columns]
        # Rename columns by combining table name and existing column name
        new_columns = [f"{table_name}_{col}" for col in df.columns]
        df.columns = new_columns
        
        # Merge or join with the existing merged_data
        merged_data = pd.merge(merged_data, df, how='outer', left_index=True, right_index=True)

    # Close the database connection
    conn.close()

# If you want to fill NaN values with a specific value, you can use:
# merged_data = merged_data.fillna(your_value)

# Print or use the merged_data DataFrame
#print(merged_data)


In [11]:
import numpy as np

df2 = merged_data.copy()

# Define the features (independent variables) and target (dependent variable)
features = list(df2)

# Define the lag order (number of lags to create)
lag_order = 6  # Example: Creating lagged values for the past n periods

for feature in features:
    for lag in range(1, lag_order):
        df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)


# replace inf and -inf with na and  drops rows with missing values in the lagged variables
#df2 = df2.replace([np.inf, -np.inf], np.nan).dropna()


C:\Users\dstoc\AppData\Local\Temp\ipykernel_22292\3632013898.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_22292\3632013898.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_22292\3632013898.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [14]:
import pickle
AllVar_weekly = df2.copy()
# Serialize the object using pickle and save to a file
with open('AllVar_weekly.pkl', 'wb') as file:
    pickle.dump(AllVar_weekly, file)



In [ ]:

# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target (created by the shift)
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR_lead'])